In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

In [ ]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
Base = automap_base()
Base.prepare(engine, reflect=True)

# reflect the tables


In [ ]:
# We can view all of the classes that automap found
#use inspect function to get classes within engine
inspector = inspect(engine)
inspector.get_table_names()

In [ ]:
# Save references to each table
measurement = Base.classes.measurement
station = Base.classes.station

In [ ]:
 # Use Inspector to print the column names and types
columns = inspector.get_columns('measurement')
for c in columns:
    print(c['name'], c["type"])

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Climate Analysis

In [ ]:
# Save the query results as a Pandas DataFrame and set the index to the date column
precipitation_data = pd.DataFrame(engine.execute('SELECT date, prcp FROM measurement ORDER BY date DESC').fetchall())
precipitation_data.columns = ["date","prcp"]
precipitation_data["date"] = pd.to_datetime(precipitation_data["date"], format = '%Y-%m-%d')
start_date = '2016-8-23'
precipitation_data = precipitation_data.loc[precipitation_data["date"] > start_date]
precipitation_data = precipitation_data.set_index("date")

# Use Pandas Plotting with Matplotlib to plot the data
plt.plot(precipitation_data.index,precipitation_data["prcp"])
plt.ylabel("Precipitation")
plt.xlabel("Date")
plt.xticks(rotation=90)
plt.title("Precipitation by Date")

plt.show()

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
precipitation_data.describe()

In [ ]:
# What are the most active stations? (i.e. what stations have the most rows)?
# List the stations and the counts in descending order.
engine.execute('SELECT station, COUNT(id) AS observation FROM measurement GROUP BY station ORDER BY observation DESC').fetchall()

In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature of the most active station?
print("Min Temperature") 
print(engine.execute('SELECT MIN(tobs) FROM measurement WHERE station = "USC00519281"').fetchall())
print("------")
print("Max Temperature")
print(engine.execute('SELECT MAX(tobs) FROM measurement WHERE station = "USC00519281"').fetchall())
print("------")
print("Average Temperature")
print(engine.execute('SELECT AVG(tobs) FROM measurement WHERE station = "USC00519281"').fetchall())

In [ ]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
#print(engine.execute('SELECT station, COUNT(id) AS observation FROM measurement WHERE tobs IS NOT NULL GROUP BY station ORDER BY observation DESC').fetchall())
temp_data = pd.DataFrame(engine.execute('SELECT id,date,tobs FROM measurement WHERE station="USC00519281"').fetchall())
temp_data.columns = ["id","date","tobs"]

temp_data["date"] = pd.to_datetime(temp_data["date"], format = '%Y-%m-%d')
start_date = '2016-8-23'
temp_data = temp_data.loc[temp_data["date"] > start_date]
plt.hist(temp_data["tobs"], bins=12)
plt.xlabel = ('Temperature Observed')
plt.ylabel = ("Frequence of Observation")
plt.title = ("Distribution of Observted Temperature for USC00519281")


## Bonus Challenge Assignment

In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# function usage example
print(calc_temps('2012-02-28', '2012-03-05'))

In [ ]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.


In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)


In [ ]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation



In [ ]:
# Create a query that will calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()
    
daily_normals("01-01")

In [ ]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`

# Set the start and end date of the trip

# Use the start and end date to create a range of dates

# Stip off the year and save a list of %m-%d strings

# Loop through the list of %m-%d strings and calculate the normals for each date


In [ ]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index


In [ ]:
# Plot the daily normals as an area plot with `stacked=False`
